# <font size=6>Trabalho Final de Algoritmos Genéticos

___
<font size=3>Este arquivo armazena as informações sobre o trabalho semestral desenvolvido na disciplina de **redes neurais e algoritmos genéticos por alunos da Ilum Escola de Ciência.**</font>

___

<font size=3>**Alunos:** Cauê Gomes Correia dos Santos, Izaque Junior Oliveira Silva e Karla Rovedo Pascoalini.</font> 

<font size=3>**Professor:** Daniel Roberto Cassar.</font> 
___

<font size=3>O intuito deste trabalho é desenvolver um `Algoritmo Genético` que otimize a função objetivo `energiaMin`, que busca a menor energia de um conjunto de átomos qualquer. De acordo com o método de Huckel é possível calcular todas as energias possíveis a partir de todas as combinações possíveis dado apenas o número de átomos. A forma encontrada pelo aluno de como essa função calcula a menor energia tende a ser custosa computacionalmente, como se pode imaginar. Devido a esse problema, resolveu-se aplicar um algoritmo genético, de minimização, para que o custo computacional seja menor e que possamos obter sem testar todas as possibilidade, uma das menores energias do conjunto de átomos. Com o intuito de tornar mais claro o fato do código feito pelo aluno Cauê ser muito custoso computacionalmente, vamos demonstrar a equação que calcula a quantidade total de conformações possívels da molécula dado apenas o número de átomos:
</font>

$$\text{c_Total} = 2^{\frac{n(n-1)}{2}}$$

<font size=3>Onde cT é u número de Conformações Totais e n o número de átomos. Note que quando olhamos 
    
    Para um conjunto de 2 átomos, cT é igual a 2. 
    
    Para n = 5, cT = 1024.
    
    Para n = 7, cT = 2.09 x 10^(6) ou ~2 milhões.
    
    Para n = 10, cT = 3.51 x 10^(13) ou ~35 trilhões.
    
Ou seja, cada átomo adicionado aumenta exponencialmente a quantidade de conformações totais. E isso é muito custoso. Foi necessário a utilização do HPC para calcular mais do que 7 átomos dado o poder computacional exigido.

## 1 - Importando as bibliotecas
___ 

In [1]:
import random
import numpy as np
from scripts import criaTriu
from scripts import testaMatriz
from scripts import energiaMin
from scripts import fitness as funcao_objetivo
from scripts import iniciaPopu
from scripts import selecaoTorneio
from scripts import cruzamento
from scripts import mutaTriu

Todas as funções importadas do scripts são funções feitas pelos alunos que são etapas cruciais do projeto de algoritmos genéticos.

## 2 - Código inicial não-otimizado
### "Nobody knows me like you do" - Made For Me, Muni Long.
___

<font size=3>A função `energiaMin` recebe a quantidade de átomos que estarão presentes na molécula. A partir desses dados ela calcula a quantidade de possíveis arranjos entre esse átomos e gera todas as possíveis matrizes, que são `binárias`. Com as matrizes geradas na função acima, a função `testaMatriz` recebe as matrizes e calcula a energia total (`t_total`) de cada matriz e rotorna a `melhor matriz`, junto a seus `autovalores (em ordem crescente)` e a `menor energia encontrada`.</font>

In [2]:
matrizes = energiaMin(6)
A = testaMatriz(matrizes)

------------------------------------------------------------ Resultado ------------------------------------------------------------
A matriz com o menor t_total associado é:
[[ 0.  0. -1. -1.  0. -1.]
 [ 0.  0.  0. -1. -1. -1.]
 [-1.  0.  0.  0. -1. -1.]
 [-1. -1.  0.  0.  0. -1.]
 [ 0. -1. -1.  0.  0. -1.]
 [-1. -1. -1. -1. -1.  0.]]
Com os autovalores [-3.4494897427831814, -0.6180339887498958, -0.6180339887498947, 1.4494897427831785, 1.6180339887498956, 1.6180339887498956]
E com o valor associado de -9.371115440565944


## 3 - Código atual otimizado
### "O acaso vai me proteger" - Epitáfio, Titãs
___

<font size=3>Na tentativa de resolver o problema de encontrar a menor energia com um custo computacional menor, desenvolvemos um código de algoritmo genético para otimizar o nosso objetivo, cuja `função objetivo` é o cálculo da energia total. Ao final, o algoritmo nos entregará o indivíduo com a menor `energia total` encontrada. É importante comentar que não necessáriamente o código devolve o mínimo global, ou seja a menor eneriga de todas as conformações, mas devolve a menor energia dadas as opções que ele seleciona.</font>

<font size=3>Nosso código contou com alterações nos operadores genéticos `cruzamento` e `mutação`, os quais adaptamos para resolver o nosso problema. Os operadores utilizados foram:</font>

<font size=3>- Seleção: Seleção por torneio com 3 indivíduos.</font>


<font size=3>- Cruzamento: Cruzamento de ponto simples adaptado a matrizes altera apenas a parte superior da matriz visto que ela é uma matriz triangular superior.</font>


<font size=3>- Mutação: Mutação simples e binária, porém com a restrição de somente ocorrer fora e acima da diagonal principal.</font>


<font size=3>- Hall da Fama: Indivíduo (Hamiltoniano) com a menor energia total encontrada.</font>

In [7]:
def genetic_algorithm(pop_size, matrix_size, geracoes, chanceMutacao, chanceCruzamento):
    # Initialize populacao
    populacao = iniciaPopu(pop_size, matrix_size)
    
    for geracao in range(geracoes):
        # Evaluate fitness of the populacao
        valoresFit = [funcao_objetivo(hamiltoniano) for _, hamiltoniano in populacao] #[6]
        
        novaPop = []
        
        # Generate new populacao
        for _ in range(pop_size // 2):
            # Select parents
            pai1 = selecaoTorneio(populacao, valoresFit)
            pai2 = selecaoTorneio(populacao, valoresFit)
            
            # cruzamento
            child1, child2 = cruzamento(pai1, pai2, chanceCruzamento)
            
            
            # Mutate
            lista_filhos = [child1[0], child2[0]]
            
            mutaTriu(lista_filhos, chanceMutacao) #[13]
            
            novaPop.extend([child1, child2])
        
        # Replace old populacao with new populacao
        populacao = novaPop
    
    # Final populacao fitness evaluation
    valoresFit = [funcao_objetivo(hamiltoniano) for _, hamiltoniano in populacao] #[6]
    ganhador = populacao[np.argmin(valoresFit)] # [4]
    
    return ganhador, valoresFit

Criando uma população e aplicando no algoritmo genético [3] para encontrar uma das (se não a) conformações com a melhor minimização de energia:

In [6]:
pop_size = 100
matrix_size = 6
geracoes = 80
chanceCruzamento = 0.5
chanceMutacao = 0.2

melhorMatriz, fit = genetic_algorithm(pop_size, matrix_size, geracoes, chanceMutacao, chanceCruzamento)

print("Best solution found:")
print(melhorMatriz[1])
print("Fitness of the best solution:", funcao_objetivo(melhorMatriz[1]))

Best solution found:
[[0. 1. 1. 1. 0. 0.]
 [1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 1. 0. 0. 1.]
 [0. 0. 1. 1. 1. 0.]]
Fitness of the best solution: -9.371115440565935


Aqui recebemos uma matriz, que é a configuração do hamiltoniano com a melhor energia minimizada e logo abaixo a função objetivo dela, que é a menor possível encontrada pelo GA, de -9.37.

Apesar da energia estar negativa, isso não é um problema pois é apenas questão de referencial.

## Referências Bibliográficas
<br>[1] Cassar, Daniel Roberto. GA 4.2 - Notebook Descobrindo a senha.</br>
<br>[2]Santos, C. e Lima, F. - Hamiltonian for n-site molecules </br>
<br>[3] Cassar, Daniel Roberto. ATP-303 GA 2.3 - Notebook algoritmo genético</br> 
<br>[4] numpy.org - Função ArgMin do Numpyhttps: //numpy.org/doc/stable/reference/generated/numpy.argmin.html </br>
<br>[5] docs.python.org - Função Format e Zfill: https://docs.python.org/pt-br/3/tutorial/inputoutput.html </br>
<br>[6] Alura - Desempacotando tuplas: https://www.alura.com.br/artigos/entendendo-o-desempacotamento-no-python </br>
<br>[7] Libre Texts - 4.13C: Hückel MO Theory: https://chem.libretexts.org/Bookshelves/Inorganic_Chemistry/Map%3A_Inorganic_Chemistry_(Housecroft)/04%3A_Experimental_Techniques/4.13%3A_Computational_Methods/4.13C%3A_Huckel_MO_Theory</br>
<br>[8] Cercomp.ufg - Huckel.ppt: https://files.cercomp.ufg.br/weby/up/56/o/Quantica_-_metodo_huckel_visao_geral.pdf</br>
<br>[9] PrOgRaMaNdO - 08 - Algoritmos Genéticos - Métodos de Seleção de Indivíduos: https://www.youtube.com/watch?v=zQLuryIF-pU</br>
<br>[10] StackOverFlow.com - How to write LaTeX in IPython Notebook?: https://stackoverflow.com/questions/13208286/how-to-write-latex-in-ipython-notebook</br>
<br>[11] GeeksForGeeks - Python | random.sample() function : https://www.geeksforgeeks.org/python-random-sample-function/</br>
<br> [12] Fernando dos Santos - Algoritmos Genéticos: Roleta (vídeo extra) : https://www.youtube.com/watch?v=c2uPOs3VawU </br><br> [13] numpy.org - Numpy.triu: https://numpy.org/doc/stable/reference/generated/numpy.triu.html </br>